In [1]:
import itertools
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.tree._tree import TREE_LEAF
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, balanced_accuracy_score, roc_auc_score, accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import ttest_ind
from tqdm import tqdm

In [2]:
compas = pd.read_csv("compas-scores-two-years.csv")
compas

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,10996,steven butler,steven,butler,2013-11-23,Male,1992-07-17,23,Less than 25,African-American,...,5,Medium,2013-11-23,2013-11-22,2013-11-24,0,1,860,0,0
7210,10997,malcolm simmons,malcolm,simmons,2014-02-01,Male,1993-03-25,23,Less than 25,African-American,...,5,Medium,2014-02-01,2014-01-31,2014-02-02,0,1,790,0,0
7211,10999,winston gregory,winston,gregory,2014-01-14,Male,1958-10-01,57,Greater than 45,Other,...,1,Low,2014-01-14,2014-01-13,2014-01-14,0,0,808,0,0
7212,11000,farrah jean,farrah,jean,2014-03-09,Female,1982-11-17,33,25 - 45,African-American,...,2,Low,2014-03-09,2014-03-08,2014-03-09,3,0,754,0,0


In [3]:
# copy the pre-processing steps from "Loading Data" found on: https://github.com/propublica/compas-analysis/blob/master/Compas%20Analysis.ipynb
compas = compas[(compas["days_b_screening_arrest"] <= 30) & (compas["days_b_screening_arrest"] >= -30)]

# nr of rows match those in link
compas["sex"]

0         Male
1         Male
2         Male
5         Male
6         Male
         ...  
7209      Male
7210      Male
7211      Male
7212    Female
7213    Female
Name: sex, Length: 6172, dtype: object

In [4]:
# separate labels
compas_y = compas["two_year_recid"]
compas_X = compas.drop("two_year_recid", axis=1)

In [5]:
compas_y = compas_y.map({0:1, 1:0})

In [6]:
compas_X = compas_X[["age", "c_charge_degree", "age_cat", "score_text", "sex", "priors_count", 
                    "days_b_screening_arrest", "decile_score", "race", "in_custody", "out_custody"]]


In [7]:
# convert dates to just years and numerical types
def date_to_justyear(date):
    if type(date) == str:
        return int(date[:4])
    
    return date

for column in ["in_custody", "out_custody"]:
    compas_X[column] = compas_X[column].apply(func=date_to_justyear, convert_dtype=True)
    

In [8]:
# separate sensitive attributes
compas_sex = compas_X["sex"]
compas_race = compas_X["race"]
compas_age = compas_X["age"]
compas_age_cat = compas_X["age_cat"]
compas_X = compas_X.drop(["race", "sex", "age", "age_cat"], axis=1)

In [9]:
compas_race = compas_race.map({"Caucasian": "White", "African-American": "Non_White", "Hispanic": "Non_White", "Other": "Non_White", "Asian": "Non_White", "Native American": "Non_White"})

In [10]:
compas_sexrace = pd.concat([compas_sex, compas_race], axis=1)
compas_sex_race = compas_sexrace[['sex', 'race']].agg('-'.join, axis=1)
compas_sex_race

0         Male-Non_White
1         Male-Non_White
2         Male-Non_White
5         Male-Non_White
6             Male-White
              ...       
7209      Male-Non_White
7210      Male-Non_White
7211      Male-Non_White
7212    Female-Non_White
7213    Female-Non_White
Length: 6172, dtype: object

In [11]:
# impute the numerical missing values with the median
compas_X = compas_X.fillna(compas_X.median())

/var/folders/d6/97vsb8ys2xsd5zzsdl4ndt6c0000gn/T/ipykernel_54173/1967445180.py:2: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  compas_X = compas_X.fillna(compas_X.median())


In [12]:
# convert to one-hot-encoding
compas_cat_X = pd.get_dummies(compas_X, columns=["in_custody", "out_custody", "c_charge_degree", "score_text"])
compas_cat_X

,priors_count,days_b_screening_arrest,decile_score,in_custody_2003,in_custody_2009,in_custody_2013,in_custody_2014,in_custody_2015,in_custody_2016,out_custody_2013,out_custody_2014,out_custody_2015,out_custody_2016,out_custody_2020,c_charge_degree_F,c_charge_degree_M,score_text_High,score_text_Low,score_text_Medium
0,0,-1.0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0
1,0,-1.0,3,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0
2,4,-1.0,4,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0
5,0,0.0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0
6,14,-1.0,6,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,0,-1.0,7,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1
7210,0,-1.0,3,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0
7211,0,-1.0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0
7212,3,-1.0,2,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0


In [13]:
# make a train and test split with same proportional size as Adult dataset
compas_train_cat_X, compas_test_cat_X, compas_train_y, compas_test_y= train_test_split(compas_cat_X, compas_y, test_size=1/3, random_state=42)

# also for the sensitive attributes with same random_state
compas_train_sex, compas_test_sex, compas_train_y, compas_test_y = train_test_split(compas_sex, compas_y, test_size=1/3, random_state=42)
compas_train_race, compas_test_race, compas_train_y, compas_test_y = train_test_split(compas_race, compas_y, test_size=1/3, random_state=42)
compas_train_age, compas_test_age, compas_train_y, compas_test_y = train_test_split(compas_age, compas_y, test_size=1/3, random_state=42)
compas_train_sex_race, compas_test_sex_race, compas_train_y, compas_test_y = train_test_split(compas_sex_race, compas_y, test_size=1/3, random_state=42)


In [14]:
# reindex all datasets, drop=True to prevent addition of "index" column
compas_train_cat_X = compas_train_cat_X.reset_index(drop=True)
compas_train_y = compas_train_y.reset_index(drop=True)
compas_train_sex = compas_train_sex.reset_index(drop=True)
compas_train_race = compas_train_race.reset_index(drop=True)
compas_train_sex_race = compas_train_sex_race.reset_index(drop=True)
compas_train_age = compas_train_age.reset_index(drop=True)

compas_test_cat_X = compas_test_cat_X.reset_index(drop=True)
compas_test_y = compas_test_y.reset_index(drop=True)
compas_test_sex = compas_test_sex.reset_index(drop=True)
compas_test_race = compas_test_race.reset_index(drop=True)
compas_test_sex_race = compas_test_sex_race.reset_index(drop=True)
compas_test_age = compas_test_age.reset_index(drop=True)

## PAFER

In [15]:
def oracle(dataset, sens_dataset, rule, s_i, mechanism=None, epsilon=0.05, delta=0.001):
    """Returns some (differentially privatised) statistics on the sensitive attribute for the specified dataframe and rule
    
    Args:
        df: The DataFrame that the developers own, which does not contain sensitive attributes.
            Used to calculate total quantities in (root) nodes.
        sens_dataset: A Series that the developers do not own, which contains the sensitive attributes. 
            Combined sensitive attributes should be encoded as a Series, e.g. Black-Female
        rule: The rule, as a string, for which the to estimate the sensitive attribute.
        s_i: The sensitive attribute, its name comes from the ith element in the set S of sensitive attributes.
            s_i should be in sens_dataset and should thus be a string. 
        mechanism: The privacy mechanism used on the returned counts. Can be one of "gaussian", "laplacian", "exponential", None. 
        epsilon: The privacy budget. Should be larger than 0.
        delta: The privacy margin. Ignored when mechanism is either laplacian or gaussian. Should be in (0, 1]. 
        
    Returns:
        The number of times s_i occurs in sens_dataset, privatised via the mechanism. 
        """
        
    # check epsilon and delta parameters
    if epsilon <= 0 or (mechanism == "gaussian" and (delta <= 0 or delta > 1 or epsilon > 1)):
        raise ValueError("The value of delta should be in (0,1] when using the gaussian mechanism")
    
    if not sens_dataset.isin([s_i]).any():
        raise KeyError("The requested sensitive attribute (s_i) is not in the sensitive dataframe (sens_dataset)")
        
    # the answer if no privacy mechanism is applied
    try:
        no_mechanism = sens_dataset.loc[dataset[pd.eval(rule, engine='python')].index].value_counts(sort=False)[s_i]
        
    except KeyError:
        no_mechanism = 0
    
    if mechanism == "laplacian":
        # this is a histogram query so the l1-sensitivity = 1 as per Dwork & Roth 
        sensitivity = 1
        return no_mechanism + np.random.laplace(loc=0, scale=sensitivity / epsilon)
    
    elif mechanism == "gaussian":
        # this is a histogram query so the l2-sensitivity = 2 as per Dwork & Roth
        sensitivity = 2
        return no_mechanism + np.random.normal(loc=0, scale=2 * sensitivity**2 * np.log(1.25 / delta) / epsilon**2)
    
    elif mechanism == "exponential":
        # this query can only change by 1 if an instance is omitted so l1-sensitivity = 1
        sensitivity = 1
        
        # np.arange is [start, stop) so + 1 for entire possible range
        possible_values = np.arange(0, sens_dataset.loc[dataset[pd.eval(rule, engine='python')].index].value_counts().to_numpy().sum() + 1)
        
        # the utility is higher when the value is closer to the actual value
        utility_scores = np.array([no_mechanism - abs(no_mechanism - value) for value in possible_values]) / 100
        probabilities = [np.exp(epsilon * score / (2 * sensitivity)) for score in utility_scores]
        
        # normalize probabilties to sum to 1
        probabilities /= np.linalg.norm(probabilities, ord=1)
        return np.random.choice(possible_values, p=probabilities)

    # if no mechanism is given, return the unprivatised cocunt
    return no_mechanism


In [30]:
def statistical_parity(y_pred, sens_dataset):
    """Calculates Statistical Parity Ratio using the predictions and the actual sensitive feature values
    
    Args:
        y_pred: The predictions, should be of same size as sens_dataset.
        sens_dataset: The Series with the sensitive attributes.
        
    Returns:
        The true statistical parity ratio.
        """
    accept_rates = []
    
    for sens_attr in sorted(sens_dataset.unique()):
        accept_rates.append(np.sum((sens_dataset == sens_attr) & y_pred) / np.sum(sens_dataset == sens_attr))
        
    return min(accept_rates) / max(accept_rates)


def estimate_sp(pos_ruleset, dataset, sens_dataset, S, mechanism, epsilon, delta=0.001):
    """Returns the estimated Statistical Parity of a tree for a privacy mechanism
    
    Args:
        pos_ruleset: A list of rules that classify favorably in the tree. This is the representation of the
        (relevant parts of the) tree. 
        dataset: The DataFrame that the developers own that does not contain sensitive feature values.
        sens_dataset: The Series that contains the sensitive features, which the developers do not own.
        S: The set/list of sensitive attributes, should all be in the sens_dataset attribute.
        mechanism: The mechanism with which to privatise the queries. 
        epsilon: The privacy budget for the privacy mechanism. Should be larger than 0.
        delta: The privacy margin. Ignored when mechanism is either laplacian or gaussian. Should be in (0, 1].
        
    Returns:
        The statistical parity ratio for the specified pos_ruleset. 
        """
    
    poscounts_per_si = np.zeros(len(S))
    
    # the variable name of the current dataset is inferred from the ruleset
    datasetname = str(pos_ruleset[0].split('[')[0])[1:]
    
    # the base rule is a rule that includes all individuals, i.e. the condition is a tautology
    # in this case we select all rows that have a value that is in the set of possible values of the first column
    base_rule = f"({datasetname}[{datasetname}.columns[0]].isin({datasetname}[{datasetname}.columns[0]].unique()))"
    
    # query the size of each sensitive attribute in the dataset
    total_per_si = [oracle(dataset, sens_dataset, base_rule, s_i, mechanism, 0.5 * epsilon, delta) for s_i in S]
    
    # replace each invalid value with balanced totals
    for i, tot in enumerate(total_per_si):
        if tot < 0 or tot > len(sens_dataset):
            total_per_si[i] = (1 / len(S)) * len(sens_dataset)
        
    total_per_si = np.array(total_per_si)
    
    for rule in pos_ruleset:
        # for each rule we find the distribution of sensitive attributes
        rule_counts = np.zeros(len(S))
        rule_total = len(sens_dataset[pd.eval(rule)])
        
        for i, s_i in enumerate(S):
            # because the queries are disjoint, epsilon remains equal across queries
            answer = round(oracle(dataset, sens_dataset, rule, s_i, mechanism, 0.5 * epsilon, delta))

            # if invalid answers from query: replace with balanced node value
            if answer < 0 or answer > len(sens_dataset):
                answer = (1 / len(S)) * rule_total

            rule_counts[i] += answer
        
        # the distribution for the current rule is added to the total
        poscounts_per_si += rule_counts
    
    # calculate and return sp
    accept_rates = poscounts_per_si / total_per_si
    return np.min(accept_rates) / np.max(accept_rates)


## Tree construction pipeline

In [17]:
tree = DecisionTreeClassifier(random_state=42)
parameter_grid={"criterion":["entropy", "gini"], 
                "max_features":["sqrt", None, "log2"], "ccp_alpha": [0.01, 0.05, 0.1],
               "min_impurity_decrease": [0.001, 0.005, 0.01, 0.025, 0.05], 
                "min_samples_leaf":[0.01, 0.02, 0.025, 0.05, 0.075, 0.1]}

tree_cv = GridSearchCV(tree, param_grid=parameter_grid, scoring='balanced_accuracy', n_jobs=2, cv=3, verbose=1)
tree_cv.fit(compas_train_cat_X, compas_train_y)
best_tree = tree_cv.best_estimator_

Fitting 3 folds for each of 540 candidates, totalling 1620 fits


In [18]:
def find_paths(tree, dataset):
    def find_path(node_numb, path, x):
        path.append(node_numb)
        if node_numb == x:
            return True
        left = False
        right = False
        if (tree.tree_.children_left[node_numb] !=-1):
            left = find_path(tree.tree_.children_left[node_numb], path, x)
        if (tree.tree_.children_right[node_numb] !=-1):
            right = find_path(tree.tree_.children_right[node_numb], path, x)
        if left or right:
            return True
        path.remove(node_numb)
        return False

    # Leaves
    leave_id = tree.apply(dataset)

    paths = {}
    for leaf in np.unique(leave_id):
        path_leaf = []
        find_path(0, path_leaf, leaf)
        paths[leaf] = np.unique(np.sort(path_leaf))
        
    return paths
        
def avg_feature_thresholds(tree, dataset, max_depth=7):
    paths = find_paths(tree, dataset)
    feature_clause_thresholds = np.zeros((dataset.shape[1], max_depth))
    feature_clause_counts = np.zeros((dataset.shape[1], max_depth))
    seen_nodes = set()

    for path in paths.values():
        # last node is a leaf node that contains no feature
        for depth, node in enumerate(path[:-1]):
            if node not in seen_nodes:
                feature_clause_thresholds[tree.tree_.feature[node], depth] += tree.tree_.threshold[node]
                feature_clause_counts[tree.tree_.feature[node], depth] += 1
                seen_nodes.add(node)
    
    return feature_clause_thresholds, feature_clause_counts

find_paths(best_tree, compas_train_cat_X)

{2: array([0, 1, 2], dtype=int64),
 3: array([0, 1, 3], dtype=int64),
 6: array([0, 4, 5, 6], dtype=int64),
 7: array([0, 4, 5, 7], dtype=int64),
 8: array([0, 4, 8], dtype=int64)}

In [19]:
def find_best_forest(n_trees=50, max_depth=7):
    # Train a random forest classifier with n_estimators trees
    rf = RandomForestClassifier(random_state=42, n_estimators=n_trees)
    parameter_grid={"criterion":["entropy", "gini"], "max_depth":np.arange(3,7+1), 
                    "max_features":["sqrt", None], 
                    "min_samples_leaf":[0.01, 0.025, 0.05, 0.075, 0.1], 
                    "ccp_alpha": np.logspace(-5, 0, 5)[:-1]}

    rf_cv = GridSearchCV(rf, param_grid=parameter_grid, scoring='balanced_accuracy', n_jobs=3, cv=3, verbose=1)
    rf_cv.fit(compas_train_cat_X.values, compas_train_y)
    return rf_cv.best_estimator_
    
def round_half(num):
    return round(num * 2) / 2.0


In [20]:
# transform the most useful numerical features to categorical features using binning
n_trees = 5
max_depth = 7
numeric_feature_idxs = [0,1,2,3]
best_rf = find_best_forest(n_trees=n_trees, max_depth=max_depth)

# calculate average over all trees
feature_clause_thresholds = np.zeros((compas_train_cat_X.shape[1], max_depth))
feature_clause_counts = np.zeros((compas_train_cat_X.shape[1], max_depth)) 
for tree in best_rf.estimators_:
    t, c = avg_feature_thresholds(tree, compas_train_cat_X, max_depth=max_depth)
    feature_clause_thresholds += t
    feature_clause_counts += c 

# categorize the numeric variables
useful_feature_idxs = []
for numeric_feature_idx in numeric_feature_idxs:
    thresholds, counts = feature_clause_thresholds[numeric_feature_idx], feature_clause_counts[numeric_feature_idx]

    if counts.any():
        # ignore division by zero and zero / zero warnings
        with np.errstate(divide='ignore', invalid='ignore'):
            avgs = (thresholds/counts)

        bins = [-np.inf] + sorted(np.unique([round(avg) for avg in avgs[~np.isnan(avgs)]])) + [np.inf] 
        names = [f'<{bins[1]}'] + [f'{binn}-{bins[i+2]}' for i, binn in enumerate(bins[1:-2])] + [f'{bins[-2]}+']

        compas_train_cat_X.iloc[:, numeric_feature_idx] = pd.cut(compas_train_cat_X.iloc[:, numeric_feature_idx], bins, labels=names)
        compas_test_cat_X.iloc[:, numeric_feature_idx] = pd.cut(compas_test_cat_X.iloc[:, numeric_feature_idx], bins, labels=names)
        useful_feature_idxs.append(numeric_feature_idx)

compas_train_cat_X = pd.get_dummies(data=compas_train_cat_X, columns=compas_train_cat_X.columns[useful_feature_idxs])
compas_test_cat_X = pd.get_dummies(data=compas_test_cat_X, columns=compas_test_cat_X.columns[useful_feature_idxs])


Fitting 3 folds for each of 400 candidates, totalling 1200 fits


/Users/florianvandersteen/miniconda3/envs/experiments/lib/python3.11/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/Users/florianvandersteen/miniconda3/envs/experiments/lib/python3.11/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/Users/florianvandersteen/miniconda3/envs/experiments/lib/python3.11/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/Users/florianvandersteen/miniconda3/envs/experiments/lib/python3.11/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/Users/florianvandersteen/miniconda3/envs/experiments/lib/python3.11/site-packages/sklearn/base.py:413: UserWarning: X has feature names, bu

In [21]:
compas_test_cat_X

,in_custody_2003,in_custody_2009,in_custody_2013,in_custody_2014,in_custody_2015,in_custody_2016,out_custody_2013,out_custody_2014,out_custody_2015,out_custody_2016,...,priors_count_0-1,priors_count_1-2,priors_count_2-6,priors_count_6+,days_b_screening_arrest_<-1,days_b_screening_arrest_-1-0,days_b_screening_arrest_0+,decile_score_<5,decile_score_5-6,decile_score_6+
0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,1
1,0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,0,1,0,1,0,0
2,0,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,1,0,1,0,0
3,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0
4,0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053,0,0,0,0,1,0,0,0,1,0,...,0,0,1,0,1,0,0,1,0,0
2054,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2055,0,0,0,1,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,1
2056,0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,1,0,0


In [22]:
def find_best_tree(dataset, dataset_labels, minleaf=0.01):
    # no random_state because we want a different tree each run
    tree = DecisionTreeClassifier()

    parameter_grid = {"criterion":["entropy", "gini"],
                      "max_features":["sqrt", "log2"], 
                      "min_samples_leaf":[minleaf]}
    
    tree_cv = GridSearchCV(tree, param_grid=parameter_grid, scoring='balanced_accuracy', n_jobs=2, cv=3, verbose=0)
    tree_cv.fit(dataset, dataset_labels)
    best_tree = tree_cv.best_estimator_
    return best_tree


In [23]:
# taken from: https://stackoverflow.com/a/51398390
def is_leaf(inner_tree, index):
    # check whether node is leaf node
    return (inner_tree.children_left[index] == TREE_LEAF and 
            inner_tree.children_right[index] == TREE_LEAF)

def prune_index(inner_tree, decisions, index=0):
    # start pruning from the bottom - if we start from the top, we might miss
    # nodes that become leaves during pruning
    if not is_leaf(inner_tree, inner_tree.children_left[index]):
        prune_index(inner_tree, decisions, inner_tree.children_left[index])
    if not is_leaf(inner_tree, inner_tree.children_right[index]):
        prune_index(inner_tree, decisions, inner_tree.children_right[index])

    # prune children if both children are leaves now and make the same decision
    if (is_leaf(inner_tree, inner_tree.children_left[index]) and
        is_leaf(inner_tree, inner_tree.children_right[index]) and
        (decisions[index] == decisions[inner_tree.children_left[index]]) and 
        (decisions[index] == decisions[inner_tree.children_right[index]])):
        # turn node into a leaf by "unlinking" its children
        inner_tree.children_left[index] = TREE_LEAF
        inner_tree.children_right[index] = TREE_LEAF

def prune_duplicate_leaves(mdl):
    # Remove leaves if all siblings make the same decision
    decisions = mdl.tree_.value.argmax(axis=2).flatten().tolist() # Decision for each node
    prune_index(mdl.tree_, decisions)
    
# pruning happens in-place
prune_duplicate_leaves(best_tree)

In [24]:
def positive_rules (tree, rules):
    """From the extracted rules, return those that have a favorable classification

    Arg:
        tree: The tree classification object from which the rules are extracted. 
        rules: Dict of which the values are rule strings.

    Returns:
        A list of all the rules that classify favorably"""

    # only those rules are added for which the majority of individuals in the node is at index 1, i.e. max
    # index 1 corresponds to class 1
    return [rule for node_id, rule in rules.items() if np.argmax(tree.tree_.value[node_id][0])]


In [25]:
# taken from: https://stackoverflow.com/a/56427596
def extract_pos_rules(tree, dataset, datasetname):
    n_nodes = tree.tree_.node_count
    children_left = tree.tree_.children_left
    children_right = tree.tree_.children_right
    feature = tree.tree_.feature
    threshold = tree.tree_.threshold

    def find_path(node_numb, path, x):
        path.append(node_numb)
        if node_numb == x:
            return True
        left = False
        right = False
        if (children_left[node_numb] !=-1):
            left = find_path(children_left[node_numb], path, x)
        if (children_right[node_numb] !=-1):
            right = find_path(children_right[node_numb], path, x)
        if left or right :
            return True
        path.remove(node_numb)
        return False


    def get_rule(datasetname, path, column_names):
        mask = '('
        for index, node in enumerate(path):
            # check if we are not in the leaf
            if index!=len(path)-1:
                # under or over the threshold?
                if (children_left[node] == path[index+1]):
                    mask += f"{datasetname}['{column_names[feature[node]]}']<= {threshold[node]}\t "
                else:
                    mask += f"{datasetname}['{column_names[feature[node]]}']> {threshold[node]} \t "

        # insert the & at the right places
        mask = mask.replace("\t", "&", mask.count("\t") - 1)
        mask = mask.replace("\t", "")
        mask += ")"
        return mask
    
    # Leaves
    leave_id = tree.apply(dataset)

    paths = {}
    for leaf in np.unique(leave_id):
        path_leaf = []
        find_path(0, path_leaf, leaf)
        paths[leaf] = np.unique(np.sort(path_leaf))

    rules = {}
    for key in paths:
        rules[key] = get_rule(datasetname, paths[key], [name for name in dataset.columns])
        
    return positive_rules(tree, rules)
        
# extract_pos_rules(best_tree, compas_train_cat_X, "compas_train_cat_X")

## Experiments

In [26]:
def bootstrap(dataset, dataset_labels, sens_dataset):
    indices = np.random.choice(dataset.index, size=len(dataset.index))
    
    return dataset.iloc[indices], dataset_labels.iloc[indices], sens_dataset.iloc[indices]

# dataset, labels, sens_dataset = bootstrap(adult_test_cat_X, adult_test_y, adult_test_sex)

In [27]:
def experiment(trainset, sens_trainset, trainsetname, trainset_labels, testset, sens_testset, testsetname, 
               testset_labels, epsilon=0.1, minleaf=0.01, runs=5, combined=False):
    tree_sps = np.zeros(runs)
    estimated_sps = np.zeros(runs)
    for i in range(runs):
        ruleset = []
        while ruleset == [] or ruleset == ['()']:
            # sample with replacement
            dataset, dataset_labels, sens_dataset = bootstrap(trainset, trainset_labels, sens_trainset)
            
            # build tree 
            best_tree = find_best_tree(trainset, trainset_labels, minleaf)
        
            # extract positive rules
            prune_duplicate_leaves(best_tree)
            ruleset = extract_pos_rules(best_tree, trainset, testsetname)
        
        if combined:
            ruleset = [" | ".join(rule for rule in ruleset)]

        # calculate true sp
        tree_sps[i] = statistical_parity(best_tree.predict(testset), sens_testset)
        
        # apply PAFER
        estimated_sps[i] = estimate_sp(ruleset, testset, sens_testset, sorted(sens_testset.unique()), mechanism='laplacian', epsilon=epsilon)
        
    return tree_sps, estimated_sps
        
    

In [31]:
minleafs = np.linspace(0.2, 0.001, 80)
print(minleafs)
runs = 3
epsilon = 0.1

# storage for results
avg_tree_sps = []
avg_estimated_sps = []
avg_errs = []
uncerts = []
eighty_percents = []
for minleaf in tqdm(minleafs):
    tree_sps, estimated_sps = experiment(compas_train_cat_X, compas_train_sex, "compas_train_cat_X", compas_train_y, 
                                         compas_test_cat_X, compas_test_sex, "compas_test_cat_X", compas_test_y,
                                         epsilon=epsilon, minleaf=minleaf, runs=runs)
    
    avg_tree_sps.append(np.mean(tree_sps))
    avg_estimated_sps.append(np.mean(estimated_sps))
    avg_errs.append(np.mean(np.abs(tree_sps - estimated_sps)))
    uncerts.append(np.std(np.abs(tree_sps - estimated_sps)) / np.sqrt(len(tree_sps)))
    eighty_percents.append((tree_sps >= 0.8))
    

[0.2        0.19748101 0.19496203 0.19244304 0.18992405 0.18740506
 0.18488608 0.18236709 0.1798481  0.17732911 0.17481013 0.17229114
 0.16977215 0.16725316 0.16473418 0.16221519 0.1596962  0.15717722
 0.15465823 0.15213924 0.14962025 0.14710127 0.14458228 0.14206329
 0.1395443  0.13702532 0.13450633 0.13198734 0.12946835 0.12694937
 0.12443038 0.12191139 0.11939241 0.11687342 0.11435443 0.11183544
 0.10931646 0.10679747 0.10427848 0.10175949 0.09924051 0.09672152
 0.09420253 0.09168354 0.08916456 0.08664557 0.08412658 0.08160759
 0.07908861 0.07656962 0.07405063 0.07153165 0.06901266 0.06649367
 0.06397468 0.0614557  0.05893671 0.05641772 0.05389873 0.05137975
 0.04886076 0.04634177 0.04382278 0.0413038  0.03878481 0.03626582
 0.03374684 0.03122785 0.02870886 0.02618987 0.02367089 0.0211519
 0.01863291 0.01611392 0.01359494 0.01107595 0.00855696 0.00603797
 0.00351899 0.001     ]


100%|████████████████████████████████████████████████████████████████| 80/80 [09:44<00:00,  7.30s/it]


In [29]:
avg_tree_sps = np.array(avg_tree_sps)
avg_estimated_sps = np.array(avg_estimated_sps)
avg_errs = np.array(avg_errs)
uncerts = np.array(uncerts)
eighty_percents = np.array(eighty_percents)

for arr, name in zip([avg_tree_sps, avg_estimated_sps, avg_errs, uncerts, eighty_percents], ["tree_sps", "estimated_sps", "errs", "uncerts", "eighty_percents"]):
    with open(f"compas-sex-{name}", "wb") as f:
        np.save(f, arr)

In [31]:
minleafs = np.linspace(0.2, 0.001, 80)
print(minleafs)
runs = 50
epsilon = 0.1

# storage for results
avg_tree_sps = []
avg_estimated_sps = []
avg_errs = []
uncerts = []
eighty_percents = []
for minleaf in tqdm(minleafs):
    tree_sps, estimated_sps = experiment(compas_train_cat_X, compas_train_race, "compas_train_cat_X", compas_train_y, 
                                         compas_test_cat_X, compas_test_race, "compas_test_cat_X", compas_test_y,
                                         epsilon=epsilon, minleaf=minleaf, runs=runs)
    
    avg_tree_sps.append(np.mean(tree_sps))
    avg_estimated_sps.append(np.mean(estimated_sps))
    avg_errs.append(np.mean(np.abs(tree_sps - estimated_sps)))
    uncerts.append(np.std(np.abs(tree_sps - estimated_sps)) / np.sqrt(len(tree_sps)))
    eighty_percents.append((tree_sps >= 0.8))

[0.2        0.19748101 0.19496203 0.19244304 0.18992405 0.18740506
 0.18488608 0.18236709 0.1798481  0.17732911 0.17481013 0.17229114
 0.16977215 0.16725316 0.16473418 0.16221519 0.1596962  0.15717722
 0.15465823 0.15213924 0.14962025 0.14710127 0.14458228 0.14206329
 0.1395443  0.13702532 0.13450633 0.13198734 0.12946835 0.12694937
 0.12443038 0.12191139 0.11939241 0.11687342 0.11435443 0.11183544
 0.10931646 0.10679747 0.10427848 0.10175949 0.09924051 0.09672152
 0.09420253 0.09168354 0.08916456 0.08664557 0.08412658 0.08160759
 0.07908861 0.07656962 0.07405063 0.07153165 0.06901266 0.06649367
 0.06397468 0.0614557  0.05893671 0.05641772 0.05389873 0.05137975
 0.04886076 0.04634177 0.04382278 0.0413038  0.03878481 0.03626582
 0.03374684 0.03122785 0.02870886 0.02618987 0.02367089 0.0211519
 0.01863291 0.01611392 0.01359494 0.01107595 0.00855696 0.00603797
 0.00351899 0.001     ]


100%|█████████████████████████████████████████████████████████████| 80/80 [7:30:14<00:00, 337.68s/it]


In [ ]:
avg_tree_sps = np.array(avg_tree_sps)
avg_estimated_sps = np.array(avg_estimated_sps)
avg_errs = np.array(avg_errs)
uncerts = np.array(uncerts)
eighty_percents = np.array(eighty_percents)

for arr, name in zip([avg_tree_sps, avg_estimated_sps, avg_errs, uncerts, eighty_percents], ["tree_sps", "estimated_sps", "errs", "uncerts", "eighty_percents"]):
    with open(f"compas-race-{name}", "wb") as f:
        np.save(f, arr)

In [ ]:
minleafs = np.linspace(0.2, 0.001, 80)
print(minleafs)
runs = 50
epsilon = 0.1

# storage for results
avg_tree_sps = []
avg_estimated_sps = []
avg_errs = []
uncerts = []
eighty_percents = []
for minleaf in tqdm(minleafs):
    tree_sps, estimated_sps = experiment(compas_train_cat_X, compas_train_sex_race, "compas_train_cat_X", compas_train_y, 
                                         compas_test_cat_X, compas_test_sex_race, "compas_test_cat_X", compas_test_y,
                                         epsilon=epsilon, minleaf=minleaf, runs=runs)
    
    avg_tree_sps.append(np.mean(tree_sps))
    avg_estimated_sps.append(np.mean(estimated_sps))
    avg_errs.append(np.mean(np.abs(tree_sps - estimated_sps)))
    uncerts.append(np.std(np.abs(tree_sps - estimated_sps)) / np.sqrt(len(tree_sps)))
    eighty_percents.append((tree_sps >= 0.8))

In [51]:
avg_tree_sps = np.array(avg_tree_sps)
avg_estimated_sps = np.array(avg_estimated_sps)
avg_errs = np.array(avg_errs)
uncerts = np.array(uncerts)
eighty_percents = np.array(eighty_percents)

for arr, name in zip([avg_tree_sps, avg_estimated_sps, avg_errs, uncerts, eighty_percents], ["tree_sps", "estimated_sps", "errs", "uncerts", "eighty_percents"]):
    with open(f"compas-sex_race-{name}", "wb") as f:
        np.save(f, arr)